In [1]:
!pip install datasets evaluate sacrebleu

In [2]:
# Required imports
import pandas as pd
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, pipeline, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from datasets import load_dataset, DatasetDict
import evaluate

# Load the Tydi XOR dataset from Hugging Face and filter for languages
dataset = load_dataset("coastalcph/tydi_xor_rc")

# Filter the dataset for Finnish, Japanese, and Russian languages
languages = ['fi', 'ja', 'ru']
train_data = dataset["train"].filter(lambda x: x['lang'] in languages)
valid_data = dataset["validation"].filter(lambda x: x['lang'] in languages)

c:\Users\markv\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Instantiate tokenizer and model
model_name = "google/flan-t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model.to(device)

Using device: cuda


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [5]:
input_text = "Question: ビスカヤ県で初めて進出した大規模鉱業会社は何? Context: Another consequence of the Carlist defeat and ensuing abolition of the Basque institutional system was the Liberalization of the industries on the Basque Provinces, especially in Biscay. The liberalization of the mines, industries and ports attracted many companies, specially British Mining Companies, that established in Biscay along with small local societies, such as Ybarra-Mier y Compañía, creating a big industrial society, based on iron mining and industry. These expansion created very big mining companies, such as Orconera Iron Ore Company Limited and Societé Franco-Belge des Mines de Somorrostro."
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

outputs = model.generate(input_ids, max_new_tokens=128)
print(tokenizer.decode(outputs[0]))

<pad> mining</s>


In [43]:
# Preprocessing function for tokenization
def preprocess_data(batch):
    inputs = []
    targets = []

    # For each question-context-answer pair
    for question, context, lang, answer, answerable in zip(
        batch["question"], batch["context"], batch["lang"], batch["answer"], batch["answerable"]
    ):
        # Construct input as "question: <question> context: <context>"
        input_text = f"Question: {question} Context: {context}"
        inputs.append(input_text)
        target_text = answer
        targets.append(target_text)

    # Tokenize input and target pairs
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing to the train and validation sets
tokenized_train = train_data.map(preprocess_data, batched=True)
tokenized_valid = valid_data.map(preprocess_data, batched=True)

tokenized_train_shuffled = tokenized_train.shuffle(seed=10)
tokenized_valid_shuffled = tokenized_valid.shuffle(seed=10)

In [36]:
# Define metric for evaluation
sacrebleu = evaluate.load("sacrebleu")

# Custom evaluation function to compute BLEU scores
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # Decode predictions and labels
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    references = [[label] for label in decoded_labels]

    # Compute scores
    sacrebleu_score = sacrebleu.compute(predictions=decoded_preds, references=references)

    return {
        "sacrebleu": sacrebleu_score["score"]
    }

In [37]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./resultsFlanT5_trial2",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=10,
)

# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_shuffled.select(range(8)),
    eval_dataset=tokenized_valid.select(range(8)),
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Start training
trainer.train()

c:\Users\markv\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
 33%|███▎      | 2/6 [00:01<00:03,  1.23it/s]c:\Users\markv\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
                                             
 33%|███▎      | 2/6 [00:02<00:03,  1.23it/s]

{'eval_loss': 45.62352752685547, 'eval_sacrebleu': 2.2999414544192485, 'eval_runtime': 1.2134, 'eval_samples_per_second': 6.593, 'eval_steps_per_second': 1.648, 'epoch': 1.0}


                                             
 67%|██████▋   | 4/6 [00:05<00:02,  1.10s/it]

{'eval_loss': 44.80213165283203, 'eval_sacrebleu': 2.2999414544192485, 'eval_runtime': 1.1799, 'eval_samples_per_second': 6.78, 'eval_steps_per_second': 1.695, 'epoch': 2.0}


100%|██████████| 6/6 [00:06<00:00,  1.17s/it]c:\Users\markv\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
                                             
100%|██████████| 6/6 [00:09<00:00,  1.57s/it]

{'eval_loss': 44.439456939697266, 'eval_sacrebleu': 2.2999414544192485, 'eval_runtime': 1.1789, 'eval_samples_per_second': 6.786, 'eval_steps_per_second': 1.696, 'epoch': 3.0}
{'train_runtime': 9.4397, 'train_samples_per_second': 2.542, 'train_steps_per_second': 0.636, 'train_loss': 45.65257263183594, 'epoch': 3.0}


TrainOutput(global_step=6, training_loss=45.65257263183594, metrics={'train_runtime': 9.4397, 'train_samples_per_second': 2.542, 'train_steps_per_second': 0.636, 'total_flos': 4461372112896.0, 'train_loss': 45.65257263183594, 'epoch': 3.0})

In [44]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./resultsFlanT5_trial2",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=10,
)

# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_shuffled.select(range(100)),
    eval_dataset=tokenized_valid_shuffled.select(range(80)),
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Start training
trainer.train()

 13%|█▎        | 10/75 [00:06<00:44,  1.45it/s]

{'loss': 41.3672, 'grad_norm': 108.16876983642578, 'learning_rate': 2.6000000000000002e-05, 'epoch': 0.4}


 27%|██▋       | 20/75 [00:14<00:42,  1.29it/s]

{'loss': 36.9554, 'grad_norm': 126.0171890258789, 'learning_rate': 2.2e-05, 'epoch': 0.8}


 33%|███▎      | 25/75 [00:17<00:39,  1.28it/s]c:\Users\markv\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(

 33%|███▎      | 25/75 [00:29<00:39,  1.28it/s]

...

 93%|█████████▎| 70/75 [01:15<00:03,  1.29it/s]

{'loss': 28.6698, 'grad_norm': 91.4073486328125, 'learning_rate': 2e-06, 'epoch': 2.8}


100%|██████████| 75/75 [01:19<00:00,  1.30it/s]c:\Users\markv\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(

100%|██████████| 75/75 [01:32<00:00,  1.23s/it]

{'eval_loss': 27.107858657836914, 'eval_sacrebleu': 4.695020828639544, 'eval_runtime': 11.3024, 'eval_samples_per_second': 7.078, 'eval_steps_per_second': 1.77, 'epoch': 3.0}
{'train_runtime': 92.3626, 'train_samples_per_second': 3.248, 'train_steps_per_second': 0.812, 'train_loss': 32.96554321289062, 'epoch': 3.0}


TrainOutput(global_step=75, training_loss=32.96554321289062, metrics={'train_runtime': 92.3626, 'train_samples_per_second': 3.248, 'train_steps_per_second': 0.812, 'total_flos': 55767151411200.0, 'train_loss': 32.96554321289062, 'epoch': 3.0})

In [47]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./resultsFlanT5_trial2",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=10,
)

# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_shuffled,
    eval_dataset=tokenized_valid_shuffled,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Start training
trainer.train()

c:\Users\markv\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 10/4809 [00:06<56:02,  1.43it/s]

{'loss': 27.2423, 'grad_norm': 90.99745178222656, 'learning_rate': 2.9937616968184656e-05, 'epoch': 0.01}


  0%|          | 20/4809 [00:12<44:03,  1.81it/s]  

{'loss': 24.2288, 'grad_norm': 100.7487564086914, 'learning_rate': 2.9875233936369307e-05, 'epoch': 0.01}


  1%|          | 30/4809 [00:18<46:44,  1.70it/s]

...

100%|█████████▉| 4800/4809 [1:00:05<00:06,  1.40it/s]

{'loss': 0.0509, 'grad_norm': 0.41079390048980713, 'learning_rate': 5.6144728633811603e-08, 'epoch': 2.99}


100%|██████████| 4809/4809 [1:00:11<00:00,  1.59it/s]c:\Users\markv\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(

100%|██████████| 4809/4809 [1:03:23<00:00,  1.26it/s]

{'eval_loss': 0.06662019342184067, 'eval_sacrebleu': 13.536063222017306, 'eval_runtime': 191.2446, 'eval_samples_per_second': 7.216, 'eval_steps_per_second': 1.804, 'epoch': 3.0}
{'train_runtime': 3803.9463, 'train_samples_per_second': 5.055, 'train_steps_per_second': 1.264, 'train_loss': 0.5405642928353812, 'epoch': 3.0}


TrainOutput(global_step=4809, training_loss=0.5405642928353812, metrics={'train_runtime': 3803.9463, 'train_samples_per_second': 5.055, 'train_steps_per_second': 1.264, 'total_flos': 3574674405457920.0, 'train_loss': 0.5405642928353812, 'epoch': 3.0})

In [52]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./resultsFlanT5_trial2/epoch5",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=2,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=10,
)

# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_shuffled,
    eval_dataset=tokenized_valid_shuffled,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Start training
trainer.train()

c:\Users\markv\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 10/3206 [00:04<25:02,  2.13it/s]

{'loss': 0.0412, 'grad_norm': 0.2835259735584259, 'learning_rate': 2.990642545227698e-05, 'epoch': 0.01}


  1%|          | 20/3206 [00:09<25:09,  2.11it/s]

{'loss': 0.0523, 'grad_norm': 0.25933638215065, 'learning_rate': 2.9812850904553962e-05, 'epoch': 0.01}


  1%|          | 30/3206 [00:14<24:39,  2.15it/s]

...

100%|█████████▉| 3200/3206 [32:00<00:04,  1.32it/s]

{'loss': 0.0432, 'grad_norm': 0.4275466501712799, 'learning_rate': 5.6144728633811603e-08, 'epoch': 2.0}


100%|██████████| 3206/3206 [32:04<00:00,  1.50it/s]c:\Users\markv\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(

100%|██████████| 3206/3206 [35:33<00:00,  1.50it/s]

{'eval_loss': 0.06458806991577148, 'eval_sacrebleu': 16.69781380255021, 'eval_runtime': 208.1069, 'eval_samples_per_second': 6.631, 'eval_steps_per_second': 1.658, 'epoch': 2.0}
{'train_runtime': 2133.8535, 'train_samples_per_second': 6.008, 'train_steps_per_second': 1.502, 'train_loss': 0.04059957190464975, 'epoch': 2.0}


TrainOutput(global_step=3206, training_loss=0.04059957190464975, metrics={'train_runtime': 2133.8535, 'train_samples_per_second': 6.008, 'train_steps_per_second': 1.502, 'total_flos': 2383116270305280.0, 'train_loss': 0.04059957190464975, 'epoch': 2.0})

In [68]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./resultsFlanT5_trial2/epoch8",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=250,
)

# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_shuffled,
    eval_dataset=tokenized_valid_shuffled,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Start training
trainer.train()

c:\Users\markv\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  5%|▌         | 250/4809 [01:57<35:51,  2.12it/s]

{'loss': 0.0349, 'grad_norm': 0.3667924702167511, 'learning_rate': 2.8440424204616345e-05, 'epoch': 0.16}


 10%|█         | 500/4809 [03:55<33:51,  2.12it/s]

{'loss': 0.0353, 'grad_norm': 0.39768895506858826, 'learning_rate': 2.688084840923269e-05, 'epoch': 0.31}


 16%|█▌        | 750/4809 [06:34<50:41,  1.33it/s]  

...

 99%|█████████▉| 4750/4809 [49:05<00:41,  1.42it/s]

{'loss': 0.0374, 'grad_norm': 0.5027788281440735, 'learning_rate': 3.680598877105428e-07, 'epoch': 2.96}


100%|██████████| 4809/4809 [49:46<00:00,  1.61it/s]c:\Users\markv\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(

100%|██████████| 4809/4809 [52:23<00:00,  1.53it/s]

{'eval_loss': 0.06243009865283966, 'eval_sacrebleu': 18.525779406299115, 'eval_runtime': 155.2458, 'eval_samples_per_second': 8.889, 'eval_steps_per_second': 2.222, 'epoch': 3.0}
{'train_runtime': 3143.4428, 'train_samples_per_second': 6.117, 'train_steps_per_second': 1.53, 'train_loss': 0.03375381277167244, 'epoch': 3.0}


TrainOutput(global_step=4809, training_loss=0.03375381277167244, metrics={'train_runtime': 3143.4428, 'train_samples_per_second': 6.117, 'train_steps_per_second': 1.53, 'total_flos': 3574674405457920.0, 'train_loss': 0.03375381277167244, 'epoch': 3.0})

Eval on full validation dataset

In [39]:
# After training run 1
# Evaluate and collect results on validation set
# Change to tokenized_train to check on train data
eval_results = trainer.evaluate(tokenized_valid)
print('results:')
print(eval_results)

# Separate the validation dataset into answerable and unanswerable subsets
answerable_data = tokenized_valid.filter(lambda x: x["answerable"])
unanswerable_data = tokenized_valid.filter(lambda x: not x["answerable"])

# Evaluate on answerable examples
answerable_results = trainer.evaluate(answerable_data)
print('answerable:')
print(answerable_results)

# Evaluate on unanswerable examples
unanswerable_results = trainer.evaluate(unanswerable_data)
print('unanswerable:')
print(unanswerable_results)

100%|██████████| 345/345 [02:47<00:00,  2.06it/s]


results:
{'eval_loss': 44.9827880859375, 'eval_sacrebleu': 3.8745802256783297, 'eval_runtime': 168.0772, 'eval_samples_per_second': 8.211, 'eval_steps_per_second': 2.053, 'epoch': 3.0}


100%|██████████| 238/238 [01:49<00:00,  2.18it/s]


answerable:
{'eval_loss': 45.29914855957031, 'eval_sacrebleu': 4.58833907869167, 'eval_runtime': 109.6013, 'eval_samples_per_second': 8.677, 'eval_steps_per_second': 2.172, 'epoch': 3.0}


100%|██████████| 108/108 [01:01<00:00,  1.77it/s]

unanswerable:
{'eval_loss': 44.23857879638672, 'eval_sacrebleu': 0.25487000229940127, 'eval_runtime': 61.9218, 'eval_samples_per_second': 6.928, 'eval_steps_per_second': 1.744, 'epoch': 3.0}


In [49]:
# After training run 3 (3 full epochs in total)
# Evaluate and collect results on validation set
# Change to tokenized_train to check on train data
eval_results = trainer.evaluate(tokenized_valid)
print('results:')
print(eval_results)

# Separate the validation dataset into answerable and unanswerable subsets
answerable_data = tokenized_valid.filter(lambda x: x["answerable"])
unanswerable_data = tokenized_valid.filter(lambda x: not x["answerable"])

# Evaluate on answerable examples
answerable_results = trainer.evaluate(answerable_data)
print('answerable:')
print(answerable_results)

# Evaluate on unanswerable examples
unanswerable_results = trainer.evaluate(unanswerable_data)
print('unanswerable:')
print(unanswerable_results)

100%|██████████| 345/345 [02:41<00:00,  2.14it/s]


results:
{'eval_loss': 0.06662019342184067, 'eval_sacrebleu': 13.536063222017306, 'eval_runtime': 162.4178, 'eval_samples_per_second': 8.497, 'eval_steps_per_second': 2.124, 'epoch': 3.0}


100%|██████████| 238/238 [01:52<00:00,  2.11it/s]


answerable:
{'eval_loss': 0.04146098718047142, 'eval_sacrebleu': 15.893940819467097, 'eval_runtime': 113.3546, 'eval_samples_per_second': 8.39, 'eval_steps_per_second': 2.1, 'epoch': 3.0}


100%|██████████| 108/108 [01:01<00:00,  1.76it/s]

unanswerable:
{'eval_loss': 0.12326549738645554, 'eval_sacrebleu': 1.2359014136494892, 'eval_runtime': 61.9918, 'eval_samples_per_second': 6.92, 'eval_steps_per_second': 1.742, 'epoch': 3.0}


In [54]:
# After training run 4 (5 full epochs in total)
# Evaluate and collect results on validation set
# Change to tokenized_train to check on train data
eval_results = trainer.evaluate(tokenized_valid)
print('results:')
print(eval_results)

# Separate the validation dataset into answerable and unanswerable subsets
answerable_data = tokenized_valid.filter(lambda x: x["answerable"])
unanswerable_data = tokenized_valid.filter(lambda x: not x["answerable"])

# Evaluate on answerable examples
answerable_results = trainer.evaluate(answerable_data)
print('answerable:')
print(answerable_results)

# Evaluate on unanswerable examples
unanswerable_results = trainer.evaluate(unanswerable_data)
print('unanswerable:')
print(unanswerable_results)

100%|██████████| 345/345 [02:52<00:00,  2.00it/s]


results:
{'eval_loss': 0.06458806246519089, 'eval_sacrebleu': 16.69781380255021, 'eval_runtime': 173.6791, 'eval_samples_per_second': 7.946, 'eval_steps_per_second': 1.986, 'epoch': 2.0}


100%|██████████| 238/238 [02:19<00:00,  1.71it/s]


answerable:
{'eval_loss': 0.03925130516290665, 'eval_sacrebleu': 20.067011131231375, 'eval_runtime': 140.1183, 'eval_samples_per_second': 6.787, 'eval_steps_per_second': 1.699, 'epoch': 2.0}


100%|██████████| 108/108 [01:02<00:00,  1.73it/s]

unanswerable:
{'eval_loss': 0.12144031375646591, 'eval_sacrebleu': 1.231167138246746, 'eval_runtime': 63.0068, 'eval_samples_per_second': 6.809, 'eval_steps_per_second': 1.714, 'epoch': 2.0}


In [70]:
# After training run 5 (8 full epochs in total)
# Evaluate and collect results on validation set
# Change to tokenized_train to check on train data
eval_results = trainer.evaluate(tokenized_valid)
print('results:')
print(eval_results)

# Separate the validation dataset into answerable and unanswerable subsets
answerable_data = tokenized_valid.filter(lambda x: x["answerable"])
unanswerable_data = tokenized_valid.filter(lambda x: not x["answerable"])

# Evaluate on answerable examples
answerable_results = trainer.evaluate(answerable_data)
print('answerable:')
print(answerable_results)

# Evaluate on unanswerable examples
unanswerable_results = trainer.evaluate(unanswerable_data)
print('unanswerable:')
print(unanswerable_results)

100%|██████████| 345/345 [03:24<00:00,  1.69it/s]


results:
{'eval_loss': 0.06243009865283966, 'eval_sacrebleu': 18.525779406299115, 'eval_runtime': 205.1515, 'eval_samples_per_second': 6.727, 'eval_steps_per_second': 1.682, 'epoch': 3.0}


100%|██████████| 238/238 [02:11<00:00,  1.82it/s]


answerable:
{'eval_loss': 0.03843402862548828, 'eval_sacrebleu': 22.138882711139406, 'eval_runtime': 131.7506, 'eval_samples_per_second': 7.218, 'eval_steps_per_second': 1.806, 'epoch': 3.0}


100%|██████████| 108/108 [00:48<00:00,  2.21it/s]

unanswerable:
{'eval_loss': 0.11598038673400879, 'eval_sacrebleu': 1.8359183966947754, 'eval_runtime': 49.6105, 'eval_samples_per_second': 8.647, 'eval_steps_per_second': 2.177, 'epoch': 3.0}


Train examples

In [40]:
# After training run 1
sample_inputs = train_data[:20]  # Get samples for testing
inputs = tokenizer([f"Question: {q} Context: {c}" for q, c in zip(sample_inputs["question"], sample_inputs["context"])], return_tensors="pt", padding=True).to(device)

with torch.no_grad():
    outputs = model.generate(**inputs)
    generated = tokenizer.batch_decode(outputs, skip_special_tokens=True)

for i in range(20):
    print(f"Predicted: {generated[i]}")
    print(f"Reference: {sample_inputs['answer'][i]}")


Predicted: The communist-controlled PKWN
Reference: no
Predicted: mining
Reference: Ybarra-Mier y Compañía
Predicted: Roman
Reference: 21 April 753 BC
Predicted: Bilbao
Reference: Vitoria-Gasteiz
Predicted: Austro-Hungarian Navy
Reference: six
Predicted: Salvador Guillermo Allende Gossens
Reference: Salvador Guillermo Allende Gossens
Predicted: Oswald
Reference: murder
Predicted: Paris
Reference: Paris
Predicted: Dutch chartered companies often dictated that their possessions be kept as confined as possible in
Reference: 1795
Predicted: Albuquerque
Reference: Albuquerque
Predicted: Tkaid Shinkansen
Reference: 1964
Predicted: Adolf Hitler
Reference: no
Predicted: New York
Reference: New York
Predicted: Daz had been in power since 1876 and saw the occasion of the centen
Reference: 1810
Predicted: Saint Petersburg
Reference: Saint Petersburg
Predicted: Constantinople
Reference: Constantinople
Predicted: ethnic
Reference: no
Predicted: Protestantism is diverse, being more divided theologic

In [46]:
# After training run 2
sample_inputs = train_data[:20]  # Get samples for testing
inputs = tokenizer([f"Question: {q} Context: {c}" for q, c in zip(sample_inputs["question"], sample_inputs["context"])], return_tensors="pt", padding=True).to(device)

with torch.no_grad():
    outputs = model.generate(**inputs)
    generated = tokenizer.batch_decode(outputs, skip_special_tokens=True)

for i in range(20):
    print(f"Predicted: {generated[i]}")
    print(f"Reference: {sample_inputs['answer'][i]}")


Predicted: the Soviets
Reference: no
Predicted: mining
Reference: Ybarra-Mier y Compañía
Predicted: a wall
Reference: 21 April 753 BC
Predicted: Bilbao
Reference: Vitoria-Gasteiz
Predicted: a sailor
Reference: six
Predicted: a symphony
Reference: Salvador Guillermo Allende Gossens
Predicted: a police officer
Reference: murder
Predicted: asian
Reference: Paris
Predicted: Dutch chartered companies often dictated that their possessions be kept as confined as possible in
Reference: 1795
Predicted: Albuquerque
Reference: Albuquerque
Predicted: saskatchewan
Reference: 1964
Predicted: fascist uprisings
Reference: no
Predicted: coal
Reference: New York
Predicted: the statue of the Angel
Reference: 1810
Predicted: Saint Petersburg
Reference: Saint Petersburg
Predicted: Constantinople
Reference: Constantinople
Predicted: adolescence
Reference: no
Predicted: Protestantism
Reference: diverse, being more divided theologically and ecclesiastically than either the Roman Catholic Church
Predicted: agg

In [50]:
# After training run 3 (3 full epochs in total)
sample_inputs = train_data[:20]  # Get samples for testing
inputs = tokenizer([f"Question: {q} Context: {c}" for q, c in zip(sample_inputs["question"], sample_inputs["context"])], return_tensors="pt", padding=True).to(device)

with torch.no_grad():
    outputs = model.generate(**inputs)
    generated = tokenizer.batch_decode(outputs, skip_special_tokens=True)

for i in range(20):
    print(f"Predicted: {generated[i]}")
    print(f"Reference: {sample_inputs['answer'][i]}")


Predicted: Lublin
Reference: no
Predicted: no
Reference: Ybarra-Mier y Compañía
Predicted: 21 April 753 BC
Reference: 21 April 753 BC
Predicted: no
Reference: Vitoria-Gasteiz
Predicted: six
Reference: six
Predicted: Salvador Guillermo Allende Gossens
Reference: Salvador Guillermo Allende Gossens
Predicted: 45 minutes
Reference: murder
Predicted: Paris
Reference: Paris
Predicted: Dutch colonial empire
Reference: 1795
Predicted: Albuquerque
Reference: Albuquerque
Predicted: 1964
Reference: 1964
Predicted: no
Reference: no
Predicted: 96,000
Reference: New York
Predicted: 1810
Reference: 1810
Predicted: Saint Petersburg Governorate
Reference: Saint Petersburg
Predicted: Constantinople
Reference: Constantinople
Predicted: extremists within Judaism motivated by religious rather than ethnic or nationalistic beliefs
Reference: no
Predicted: no
Reference: diverse, being more divided theologically and ecclesiastically than either the Roman Catholic Church
Predicted: Thomas the Apostle and Mar Ad

In [55]:
# After training run 4 (5 full epochs in total)
sample_inputs = train_data[:20]  # Get samples for testing
inputs = tokenizer([f"Question: {q} Context: {c}" for q, c in zip(sample_inputs["question"], sample_inputs["context"])], return_tensors="pt", padding=True).to(device)

with torch.no_grad():
    outputs = model.generate(**inputs)
    generated = tokenizer.batch_decode(outputs, skip_special_tokens=True)

for i in range(20):
    print(f"Predicted: {generated[i]}")
    print(f"Reference: {sample_inputs['answer'][i]}")


Predicted: Lublin
Reference: no
Predicted: no
Reference: Ybarra-Mier y Compañía
Predicted: 21 April 753 BC
Reference: 21 April 753 BC
Predicted: Vitoria-Gasteiz
Reference: Vitoria-Gasteiz
Predicted: six
Reference: six
Predicted: Salvador Guillermo Allende Gossens
Reference: Salvador Guillermo Allende Gossens
Predicted: 45 minutes
Reference: murder
Predicted: Paris
Reference: Paris
Predicted: Dutch colonial empire
Reference: 1795
Predicted: Albuquerque
Reference: Albuquerque
Predicted: 1964
Reference: 1964
Predicted: no
Reference: no
Predicted: 96,000
Reference: New York
Predicted: 1810
Reference: 1810
Predicted: Saint Petersburg Governorate
Reference: Saint Petersburg
Predicted: Constantinople
Reference: Constantinople
Predicted: extremists within Judaism motivated by religious rather than ethnic or nationalistic beliefs
Reference: no
Predicted: no
Reference: diverse, being more divided theologically and ecclesiastically than either the Roman Catholic Church
Predicted: Thomas the Apost

In [71]:
# After training run 5 (8 full epochs in total)
sample_inputs = train_data[:20]  # Get samples for testing
inputs = tokenizer([f"Question: {q} Context: {c}" for q, c in zip(sample_inputs["question"], sample_inputs["context"])], return_tensors="pt", padding=True).to(device)

with torch.no_grad():
    outputs = model.generate(**inputs)
    generated = tokenizer.batch_decode(outputs, skip_special_tokens=True)

for i in range(20):
    print(f"Predicted: {generated[i]}")
    print(f"Reference: {sample_inputs['answer'][i]}")


Predicted: Lublin
Reference: no
Predicted: no
Reference: Ybarra-Mier y Compañía
Predicted: 21 April 753 BC
Reference: 21 April 753 BC
Predicted: Vitoria-Gasteiz
Reference: Vitoria-Gasteiz
Predicted: no
Reference: six
Predicted: Salvador Guillermo Allende Gossens
Reference: Salvador Guillermo Allende Gossens
Predicted: 45 minutes
Reference: murder
Predicted: 2,140,526
Reference: Paris
Predicted: trade-based system which derived most of its influence from merchant enterprise and from Dutch control of
Reference: 1795
Predicted: Albuquerque
Reference: Albuquerque
Predicted: 1964
Reference: 1964
Predicted: Benito Mussolini
Reference: no
Predicted: 96,000
Reference: New York
Predicted: 1810
Reference: 1810
Predicted: Saint Petersburg Governorate
Reference: Saint Petersburg
Predicted: Constantinople
Reference: Constantinople
Predicted: extremists within Judaism motivated by religious rather than ethnic or nationalistic beliefs
Reference: no
Predicted: no
Reference: diverse, being more divided

Validation examples

In [41]:
# After training run 1
sample_inputs = valid_data[:20]  # Get samples for testing
inputs = tokenizer([f"Question: {q} Context: {c}" for q, c in zip(sample_inputs["question"], sample_inputs["context"])], return_tensors="pt", padding=True).to(device)

with torch.no_grad():
    outputs = model.generate(**inputs)
    generated = tokenizer.batch_decode(outputs, skip_special_tokens=True)

for i in range(20):
    print(f"Predicted: {generated[i]}")
    print(f"Reference: {sample_inputs['answer'][i]}")


Predicted: Alec Fleming "Alex" Churchill
Reference: Colombo, British Ceylon
Predicted: Buddhism
Reference: Christianity
Predicted: Rachel
Reference: Rachel Barbra Berry
Predicted: "Killzone"
Reference: November 2004
Predicted: Pennsylvania
Reference: December 12, 1787
Predicted: Michael
Reference: Denville Hall
Predicted: 
Reference: 5th century BCE
Predicted: digital television
Reference: transmission of television signals, including the sound channel, using digital encoding
Predicted: Sami
Reference: no
Predicted: Rachel's journey of finding herself within the glee club
Reference: Lea Michele
Predicted: "Birdland"
Reference: Birdland
Predicted: drier mountains of southern Peru south to northern Chile south to about 30°S
Reference: no
Predicted: Tiziana
Reference: Lucca, Tuscany
Predicted: Takahashi
Reference: 1984
Predicted: Tang Dynasty
Reference: Tang Dynasty
Predicted: niin
Reference: nomen
Predicted: rak
Reference: anise
Predicted: machine-state functionalism
Reference: Hilary Pu

In [51]:
# After training run 3 (3 full epochs in total)
sample_inputs = valid_data[:20]  # Get samples for testing
inputs = tokenizer([f"Question: {q} Context: {c}" for q, c in zip(sample_inputs["question"], sample_inputs["context"])], return_tensors="pt", padding=True).to(device)

with torch.no_grad():
    outputs = model.generate(**inputs)
    generated = tokenizer.batch_decode(outputs, skip_special_tokens=True)

for i in range(20):
    print(f"Predicted: {generated[i]}")
    print(f"Reference: {sample_inputs['answer'][i]}")


Predicted: Alec Fleming "Alex" Churchill
Reference: Colombo, British Ceylon
Predicted: 5,8 billion people and 84% of the population
Reference: Christianity
Predicted: Rachel Barbra Berry
Reference: Rachel Barbra Berry
Predicted: series of first-person shooter and twin sticks shooter video games for Sony Computer Entertainment'
Reference: November 2004
Predicted: 1787
Reference: December 12, 1787
Predicted: Sheila Sim
Reference: Denville Hall
Predicted: no
Reference: 5th century BCE
Predicted: no
Reference: transmission of television signals, including the sound channel, using digital encoding
Predicted: no
Reference: no
Predicted: Rachel Barbra Berry
Reference: Lea Michele
Predicted: Birdland
Reference: Birdland
Predicted: no
Reference: no
Predicted: Lucca, Tuscany
Reference: Lucca, Tuscany
Predicted: Higashinaruse, Akita
Reference: 1984
Predicted: Zhou Dynasty
Reference: Tang Dynasty
Predicted: no
Reference: nomen
Predicted: rak, pastis and sambuca
Reference: anise
Predicted: Hilary P

In [56]:
# After training run 4 (5 full epochs in total)
sample_inputs = valid_data[:20]  # Get samples for testing
inputs = tokenizer([f"Question: {q} Context: {c}" for q, c in zip(sample_inputs["question"], sample_inputs["context"])], return_tensors="pt", padding=True).to(device)

with torch.no_grad():
    outputs = model.generate(**inputs)
    generated = tokenizer.batch_decode(outputs, skip_special_tokens=True)

for i in range(20):
    print(f"Predicted: {generated[i]}")
    print(f"Reference: {sample_inputs['answer'][i]}")


Predicted: Colombo, British Ceylon
Reference: Colombo, British Ceylon
Predicted: 5.8 billion people and 84% of the population
Reference: Christianity
Predicted: Rachel Barbra Berry
Reference: Rachel Barbra Berry
Predicted: series of first-person shooter and twin sticks shooter video games for Sony Computer Entertainment'
Reference: November 2004
Predicted: 13
Reference: December 12, 1787
Predicted: Sheila Sim
Reference: Denville Hall
Predicted: no
Reference: 5th century BCE
Predicted: transmission of television signals, including the sound channel, using digital encoding, in contrast to
Reference: transmission of television signals, including the sound channel, using digital encoding
Predicted: no
Reference: no
Predicted: Rachel Barbra Berry
Reference: Lea Michele
Predicted: Birdland
Reference: Birdland
Predicted: no
Reference: no
Predicted: Lucca, Tuscany
Reference: Lucca, Tuscany
Predicted: Higashinaruse, Akita
Reference: 1984
Predicted: Zhou Dynasty
Reference: Tang Dynasty
Predicted

In [72]:
# After training run 5 (8 full epochs in total)
sample_inputs = valid_data[:20]  # Get samples for testing
inputs = tokenizer([f"Question: {q} Context: {c}" for q, c in zip(sample_inputs["question"], sample_inputs["context"])], return_tensors="pt", padding=True).to(device)

with torch.no_grad():
    outputs = model.generate(**inputs)
    generated = tokenizer.batch_decode(outputs, skip_special_tokens=True)

for i in range(20):
    print(f"Predicted: {generated[i]}")
    print(f"Reference: {sample_inputs['answer'][i]}")


Predicted: Hong Kong
Reference: Colombo, British Ceylon
Predicted: 5.8 billion people
Reference: Christianity
Predicted: Rachel Barbra Berry
Reference: Rachel Barbra Berry
Predicted: 2004
Reference: November 2004
Predicted: 13
Reference: December 12, 1787
Predicted: Sheila Sim
Reference: Denville Hall
Predicted: 5th century BCE
Reference: 5th century BCE
Predicted: transmission of television signals, including the sound channel, using digital encoding, in contrast to
Reference: transmission of television signals, including the sound channel, using digital encoding
Predicted: no
Reference: no
Predicted: Rachel Barbra Berry
Reference: Lea Michele
Predicted: heavy Weather
Reference: Birdland
Predicted: no
Reference: no
Predicted: Lucca, Tuscany
Reference: Lucca, Tuscany
Predicted: 1953
Reference: 1984
Predicted: Zhou Dynasty
Reference: Tang Dynasty
Predicted: Latin "nomen", "name"
Reference: nomen
Predicted: rak, pastis and sambuca
Reference: anise
Predicted: Hilary Putnam
Reference: Hila

Looking at unanswerable validation data only

In [67]:
# After training run 4 (5 full epochs in total)
sample_inputs = valid_data.filter(lambda x: not x['answerable'])[:20]  # Get samples for testing
inputs = tokenizer([f"Question: {q} Context: {c}" for q, c in zip(sample_inputs["question"], sample_inputs["context"])], return_tensors="pt", padding=True).to(device)

with torch.no_grad():
    outputs = model.generate(**inputs)
    generated = tokenizer.batch_decode(outputs, skip_special_tokens=True)

for i in range(20):
    print(f"Questiion: {sample_inputs['question'][i]}")
    print(f"Context: {sample_inputs['context'][i]}")
    print(f"Predicted: {generated[i]}")
    print(f"Reference: {sample_inputs['answer'][i]}")


c:\Users\markv\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Questiion: Ovatko suomen kielen murteet olleet aina olemassa?
Context: Westrobothnian () is a number of closely related non-standardized Scandinavian dialects spoken natively along the coast of the historical province of Westrobothnia in co-existence with Finnish, Sami and in recent centuries, the national standard language Swedish. Westrobothnian is the northernmost dialect group of the North Germanic languages in Sweden and borders the traditional Sami-speaking Lapland to the west and Finnish-speaking Torne Valley to the north. Like all Scandinavian, the different varieties of Westrobothnian originate in Proto-Norse and dialects of Old Norse, spoken by immigrating Germanic settlers during the Viking Age.
Predicted: no
Reference: no
Questiion: Onko Andien huiput kokonaan lumen peitossa?
Context: The climate in the Andes varies greatly depending on latitude, altitude, and proximity to the sea. Temperature, atmospheric pressure and humidity decrease in higher elevations. The southern se

In [76]:
# After training run 5 (8 full epochs in total)
sample_inputs = valid_data.filter(lambda x: not x['answerable'])[:20]  # Get samples for testing
inputs = tokenizer([f"Question: {q} Context: {c}" for q, c in zip(sample_inputs["question"], sample_inputs["context"])], return_tensors="pt", padding=True).to(device)

with torch.no_grad():
    outputs = model.generate(**inputs)
    generated = tokenizer.batch_decode(outputs, skip_special_tokens=True)

for i in range(20):
    print(f"Questiion: {sample_inputs['question'][i]}")
    print(f"Context: {sample_inputs['context'][i]}")
    print(f"Predicted: {generated[i]}")
    print(f"Reference: {sample_inputs['answer'][i]}")


Questiion: Ovatko suomen kielen murteet olleet aina olemassa?
Context: Westrobothnian () is a number of closely related non-standardized Scandinavian dialects spoken natively along the coast of the historical province of Westrobothnia in co-existence with Finnish, Sami and in recent centuries, the national standard language Swedish. Westrobothnian is the northernmost dialect group of the North Germanic languages in Sweden and borders the traditional Sami-speaking Lapland to the west and Finnish-speaking Torne Valley to the north. Like all Scandinavian, the different varieties of Westrobothnian originate in Proto-Norse and dialects of Old Norse, spoken by immigrating Germanic settlers during the Viking Age.
Predicted: no
Reference: no
Questiion: Onko Andien huiput kokonaan lumen peitossa?
Context: The climate in the Andes varies greatly depending on latitude, altitude, and proximity to the sea. Temperature, atmospheric pressure and humidity decrease in higher elevations. The southern se